In [2]:
from graph2net.trainers import gen_and_validate,generate,model_validate,full_model_run, max_model_size
from graph2net.data_loaders import *
from graph2net.graph_generators import *
from graph2net.archetypes import resNeXt
from graph2net.helpers import *
from graph2net.notifier import notify_me
import gc
import logging
import numpy as np
import pandas as pd
import pickle as pkl
import time
import os

import random
import matplotlib.pyplot as plt

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)
logging.basicConfig(filename='logs/model_testbed.log', level=logging.INFO)

%load_ext autoreload
%autoreload 2

# Test Settings

In [3]:
catchup=False # or True
random_pairs=False #or True
scratch_test=False #or True

# Global Variables and Helpers

In [4]:
data = load_data(batch_size=256)

lr_schedule512 = {
    'type': 'cosine',
    'lr_min': 0,
    'lr_max': 1e-2,
    't_0': 1,
    't_mult': 2
}

def model_runner(model,epochs,prefix,drop):
    try:
        return full_model_run(model, 
                              data=data, 
                              epochs=epochs, 
                              lr=.025, 
                              momentum=.9, 
                              weight_decay=3e-4,
                              lr_schedule=lr_schedule512,
                              drop_path=drop,
                              log=True,
                              track_progress=True,
                              prefix=prefix,
                              verbose=False)
    except Exception as e:
        if "CUDA" in str(e):
            del model
            clean(verbose=False)
            raise e
        else:
            raise e

# Model Definitions

In [4]:
def general_net(cell,epochs,prefix,**kwargs):
    print("\n== {} ==".format(prefix))
    clean(None,'general init')
    t_start = time.time()
    
    reason = None
    if kwargs.get("max_size",False):
        config = max_model_size(cell,data,auxiliaries=kwargs.get('aux',[None]))
        if config:
            print("Scale: {}, Spacing: {}, Parallel: {}, Params: {:,}".format(*config))
            scale,spacing,params = config 
            reductions = 5
        else:
            model,valid,reason=False,False,"No Config"                                                 
    else:
        scale,spacing,reductions = kwargs['scale'],kwargs['spacing'],kwargs['reductions'] 
    print(scale,spacing,reductions)
    clean(None,"post config")
    print("Pre GV:",mem_stats())
    if reason is None:
        print(scale,kwargs.get('aux'),cell_space(reductions,spacing))
        model, valid, reason = gen_and_validate([cell],
                                                data,
                                                scale=scale,
                                                auxiliaries=kwargs.get('aux',[None]),
                                                cell_types=cell_space(reductions,spacing))
        clean(model,'post gv')
    if valid:
        print("Post Validation:",mem_stats())
        try:
            loss, correct, preds,acc_preds,confs = model_runner(model,epochs,prefix,drop=kwargs['drop'])
        except KeyboardInterrupt as e:
            print('Keyboard Interrupt!')
            correct = [0]
            clean(model,"general net interrupt")
            raise e
        except RuntimeError as e:
            if "CUDA" in str(e):
                print("CUDA: Out of memory")
                correct = [0]
                clean(model,"CUDA error")
            else:
                clean(model,"other Error")
                raise e
    else:
        print("Model Invalid:",reason)
        correct = [0]
                                     
    clean(model,"general net end")
    return correct,time.time()-t_start

def micro_net(cell,epochs=32):
    return general_net(cell,epochs=epochs,scale=2,spacing=0,reductions=2,prefix='Micro',drop=False)

def micro_net5(cell):
    return general_net(cell,epochs=8, scale=2,spacing=0,reductions=5,prefix='Micro5',drop=False)
    
def macro_net(cell,epochs=128):
    return general_net(cell,epochs=epochs,max_size=True,aux=[2,3,4],prefix='Macro',drop=True)

def manual_net(cell,epochs,scale,spacing,aux=[2],reductions=5,drop=True):
    return general_net(cell,epochs,scale=scale,spacing=spacing,reductions=reductions,aux=aux,prefix='Manual',drop=drop)

# Catchup Micro with Existing Macros

In [5]:
if catchup:
    cells = pkl.load(open('pickle_jar/all_macro_cells.pkl','rb'))
    cells = [x[0] if len(x)<3 else np.array(x) for x in cells]

    micro_times,micro5_times = [],[]
    for i,cell in enumerate(cells):
        print("{:>3} of {:<3}".format(i,len(cells)))
        try:
            _,micro_time = micro_net(cell)
            _,micro5_time = micro_net5(cell)
            micro_times.append(micro_time)
            micro5_times.append(micro5_time)
            print("Mean Micro Time:",np.mean(micro_times))
            print("Mean Micro5 Time:",np.mean(micro5_times))
        except KeyboardInterrupt as e:
            raise e
        except Exception as e:
            if "Input Mismatch for Summation Node" in str(e):
                print("Skipping broken cell")
            else:
                raise e

# Test Random Micro/Macro Pairs

In [6]:
if random_pairs:
    try:
        i = 0
        while 1:
            print("\n===== CELL {} ===== ".format(i))
            cell = gen_cell(np.random.randint(3,10),connectivity=np.random.uniform(.3,.7),concat=.5)        
            micro_scores,micro_time = micro_net(cell)
            micro5_scores,micro5_time = micro_net5(cell)     
            macro_scores,macro_time = macro_net(cell)
            notify_me("Test {} successful, moving on. Micro: {}, Micro5: {}, Macro: {}".format(i,max(micro_scores),max(micro5_scores),max(macro_scores)))
            i+=1
    except KeyboardInterrupt as e:
        raise e
    except Exception as e:
        notify_me("Micro/Macro test errored. {}".format(str(e)))
        raise e

# Scratch Testing Space

In [7]:
def scratch_tester1():
    cell = np.array([[0, 1, 0, 1],
            [1, 0, 9, 0],
            [0, 0, 0, 1],
            [1, 0, 0, 1]])
    macro_net(cell,epochs=1)
    clean(None,'scratch test')

def scratch_tester2():
    cell = np.array([[0, 1, 0, 1],
            [1, 0, 9, 0],
            [0, 0, 0, 1],
            [1, 0, 0, 1]])
    manual_net(cell,epochs=1,scale=7,spacing=2,aux=[2,3,4])
    clean(None,'scratch test')

if scratch_test:
    scratch_tester1()
    scratch_tester2()

# Non-deterministic weirdness

In [4]:
clean(None,None)
cell = np.array([[0, 1, 0, 1],
                [1, 0, 9, 0],
                [0, 0, 0, 1],
                [1, 0, 0, 1]])


Cleaning up at None...
Pre: 0.00B
Post: 0.00B


In [1]:
while 1:
    max_model_size(cell,data,auxiliaries=[2,3,4])
    clean(None,None)

NameError: name 'max_model_size' is not defined